# Notebook that loads a Filtered Signal and Exports it into a format that can be used by the remaining Signal-to-Spike Stages

### Check WD (change if necessary) and file loading

In [418]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Extract the channel for the 3 frequency bands

In [419]:
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

ripple_file_suffix = f'{RIPPLE_BAND_FILENAME}_band'
fr_file_suffix = f'{FR_BAND_FILENAME}_band'
hfo_file_suffix = f'{BOTH_BAND_FILENAME}_band'

In [420]:
import numpy as np
import math
from utils.io import preview_np_array

IS_CLINICAL = False
INPUT_PATH = "/clinical" if IS_CLINICAL else "/synthetic"
INPUT_FILE_NAME = f"filtered_seeg_ch90-119"     # "filtered_seeg_ics_ch30-59"   # "filtered_seeg_csl_ch30-59"   # f"filtered_seeg_ch210-239"      # 

filtered_seeg_path = f"../filter/results{INPUT_PATH}/{INPUT_FILE_NAME}"

ripple_seeg_filename = f"{filtered_seeg_path}_{ripple_file_suffix}.npy"
fr_seeg_filename = f"{filtered_seeg_path}_{fr_file_suffix}.npy"
hfo_seeg_filename = f"{filtered_seeg_path}_{hfo_file_suffix}.npy"

ripple_data = np.load(f"{PATH_TO_FILE}{ripple_seeg_filename}")
fr_data = np.load(f"{PATH_TO_FILE}{fr_seeg_filename}")
hfo_data = np.load(f"{PATH_TO_FILE}{hfo_seeg_filename}")

In [421]:
preview_np_array(ripple_data, "ripple_data", edge_items=2)

preview_np_array(fr_data, "fr_data", edge_items=2)

preview_np_array(hfo_data, "hfo_data", edge_items=2)

ripple_data Shape: (245760, 30).
Preview: [[ 4.80357903e-07  1.13618054e-08 ... -1.77857978e-06 -1.53852620e-06]
 [ 6.49123322e-06 -5.36933144e-07 ... -2.49116800e-05 -2.31873463e-05]
 ...
 [ 1.28016657e+00  4.40803823e-02 ...  3.79973658e-01 -9.67521623e-01]
 [ 1.42778692e+00 -6.26220601e-01 ...  4.49039460e-01 -1.48918836e+00]]
fr_data Shape: (245760, 30).
Preview: [[ 9.29595564e-06  2.19875303e-07 ... -3.44193332e-05 -2.97737816e-05]
 [ 5.49979186e-05 -1.20612003e-05 ... -2.20610795e-04 -2.22533492e-04]
 ...
 [ 1.05273258e-01  5.54415897e-01 ...  5.35277170e-01 -6.37674633e-01]
 [-4.14363946e-01 -4.94735956e-01 ...  7.11674086e-01 -5.66676671e-01]]
hfo_data Shape: (245760, 30).
Preview: [[ 4.09480909e-04  9.68536667e-06 ... -1.51614964e-03 -1.31151607e-03]
 [ 3.39510592e-03 -5.08286241e-04 ... -1.33184926e-02 -1.29171986e-02]
 ...
 [-7.64045105e-02 -2.77992521e-01 ...  9.25334506e-01 -1.74874841e+00]
 [-8.71713443e-02 -4.33051842e-01 ...  1.70951012e+00 -1.87446967e+00]]


## Define Global Parameters of the Experiment

In [422]:
from utils.input import SAMPLING_RATE, X_STEP

num_samples = ripple_data.shape[0]    # 2048 * 120 = 245760
num_channels = ripple_data.shape[1]   # 960

print(f"num_samples: {num_samples}, num_channels: {num_channels}")

num_samples: 245760, num_channels: 30


## Define the Channel we want to extract to feed the SNN

In [423]:
ch_idx = 5

## Fetch the signal for the specified channel

In [424]:
# Extract the seeg_channels in the range [ch_start_idx, ch_end_idx]
ripple_seeg_ch = ripple_data[:, ch_idx]
fr_seeg_ch = fr_data[:, ch_idx]
hfo_seeg_ch = hfo_data[:, ch_idx]

preview_np_array(ripple_seeg_ch, "Ripple sEEG Channel")
preview_np_array(fr_seeg_ch, "FR sEEG Channel")
preview_np_array(hfo_seeg_ch, "HFO sEEG Channel")

Ripple sEEG Channel Shape: (245760,).
Preview: [ 4.03932046e-06  5.89366126e-05  4.14094388e-04  1.86753797e-03
  6.06652167e-03 ... -6.96934261e-01 -7.47039902e-01 -7.38526318e-01
 -6.11078540e-01 -3.64342973e-01]
FR sEEG Channel Shape: (245760,).
Preview: [ 7.81695139e-05  5.46696135e-04  1.08279173e-03 -1.40195351e-03
 -8.90032230e-03 ... -4.42790739e-01 -4.09728636e-01  5.01444239e-01
  9.77458407e-01 -4.49385585e-02]
HFO sEEG Channel Shape: (245760,).
Preview: [ 0.00344332  0.0322592   0.12941021  0.27662443  0.28465375 ...
  0.34871041 -1.59826325 -2.93470791 -2.45221796 -1.1562866 ]


## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [425]:
markers_seeg_file_name = f"{filtered_seeg_path}_markers.npy"
markers = np.load(f"{PATH_TO_FILE}{markers_seeg_file_name}", allow_pickle=True)

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (30, 42)
First time steps:  [[('Spike',   1000.  , 0.) ('Spike+Fast-Ripple',   4218.75, 0.)
  ('Ripple+Fast-Ripple',   6966.8 , 0.)
  ('Ripple+Fast-Ripple',   9793.95, 0.)
  ('Spike+Ripple+Fast-Ripple',  13189.  , 0.) ...
  ('Ripple+Fast-Ripple', 107859.  , 0.) ('Spike+Ripple', 111439.  , 0.)
  ('Ripple', 114551.  , 0.) ('Spike+Ripple', 116517.  , 0.)
  ('Spike+Ripple', 119000.  , 0.)]
 [('Spike+Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   3917.97, 0.)
  ('Spike+Fast-Ripple',   6794.92, 0.) ('Fast-Ripple',   9094.73, 0.)
  ('Fast-Ripple',  12348.1 , 0.) ... ('Spike+Fast-Ripple', 107697.  , 0.)
  ('Fast-Ripple', 111079.  , 0.) ('Spike', 113382.  , 0.)
  ('Ripple+Fast-Ripple', 116656.  , 0.) ('Spike+Ripple', 119000.  , 0.)]
 [('Fast-Ripple',   1000.  , 0.) ('Spike',   3655.76, 0.)
  ('Spike',   7188.96, 0.) ('Spike',   9880.37, 0.)
  ('Spike+Ripple',  13890.6 , 0.) ...
  ('Spike+Fast-Ripple', 107924.  , 0.)
  ('Spike+Ripple+Fast-Ripple', 110238.  , 0.)
  ('Spi

### Get the markers of the specified channel

In [428]:
relevant_markers = markers[ch_idx]

preview_np_array(relevant_markers, "relevant_markers", edge_items=10)

relevant_markers Shape: (42,).
Preview: [('Spike+Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.) ('Spike',   5755.86, 0.)
 ('Ripple',   9064.45, 0.) ('Ripple+Fast-Ripple',  12262.2 , 0.)
 ('Ripple',  14958.  , 0.) ('Spike+Ripple',  18041.5 , 0.)
 ('Spike+Ripple',  21022.  , 0.) ('Fast-Ripple',  24200.7 , 0.)
 ('Spike',  26978.  , 0.) ... ('Spike+Fast-Ripple',  92921.9 , 0.)
 ('Spike+Ripple',  96429.2 , 0.) ('Spike',  99202.1 , 0.)
 ('Spike+Ripple+Fast-Ripple', 102059.  , 0.) ('Ripple', 105210.  , 0.)
 ('Spike+Fast-Ripple', 108330.  , 0.) ('Fast-Ripple', 110470.  , 0.)
 ('Spike+Fast-Ripple', 112796.  , 0.) ('Spike+Ripple', 115960.  , 0.)
 ('Ripple', 119000.  , 0.)]


## Define the Markers of each Band

In [426]:
from utils.input import label_has_fast_ripple, label_has_ripple, label_has_hfo_event

ripple_markers = []
fr_markers = []
hfo_markers = []

for marker in relevant_markers:
    curr_label = marker['label']
    if label_has_ripple(curr_label):
        ripple_markers.append(marker)
    if label_has_fast_ripple(curr_label):
        fr_markers.append(marker)
    if label_has_hfo_event(curr_label):
        hfo_markers.append(marker)

ripple_markers = np.array(ripple_markers)
fr_markers = np.array(fr_markers)
hfo_markers = np.array(hfo_markers)

In [427]:
preview_np_array(ripple_markers, "ripple_markers", edge_items=4)
preview_np_array(fr_markers, "fr_markers", edge_items=4)
preview_np_array(hfo_markers, "hfo_markers", edge_items=4)

ripple_markers Shape: (7,).
Preview: [('Ripple',  4852.539, 0.) ('Ripple',  5303.711, 0.)
 ('Ripple',  9606.445, 0.) ('Ripple', 32159.18 , 0.)
 ('Ripple', 34225.586, 0.) ('Ripple', 38057.13 , 0.)
 ('Ripple', 51099.12 , 0.)]
fr_markers Shape: (36,).
Preview: [('Fast Ripple',  3478.5156, 0.) ('Fast Ripple',  9820.3125, 0.)
 ('Fast Ripple', 11166.016 , 0.) ('Fast Ripple', 18351.562 , 0.) ...
 ('Fast Ripple', 55280.76  , 0.) ('Fast Ripple', 56544.434 , 0.)
 ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]
hfo_markers Shape: (43,).
Preview: [('Fast Ripple',  3478.5156, 0.) ('Ripple',  4852.539 , 0.)
 ('Ripple',  5303.711 , 0.) ('Ripple',  9606.445 , 0.) ...
 ('Fast Ripple', 55280.76  , 0.) ('Fast Ripple', 56544.434 , 0.)
 ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]


## Export the Processed sEEG and respective markers into a `.npy` file

In [429]:
# To keep the same format as the original data, we will add an extra dimension to the final markers
final_ripple_markers = np.array([ripple_markers])
final_fr_markers = np.array([fr_markers])
final_hfo_markers = np.array([hfo_markers])

preview_np_array(final_ripple_markers, "final_ripple_markers", edge_items=2)
preview_np_array(final_fr_markers, "final_fr_markers", edge_items=2)
preview_np_array(final_hfo_markers, "final_hfo_markers", edge_items=2)

final_ripple_markers Shape: (1, 7).
Preview: [[('Ripple',  4852.539, 0.) ('Ripple',  5303.711, 0.) ...
  ('Ripple', 38057.13 , 0.) ('Ripple', 51099.12 , 0.)]]
final_fr_markers Shape: (1, 36).
Preview: [[('Fast Ripple',  3478.5156, 0.) ('Fast Ripple',  9820.3125, 0.) ...
  ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]]
final_hfo_markers Shape: (1, 43).
Preview: [[('Fast Ripple',  3478.5156, 0.) ('Ripple',  4852.539 , 0.) ...
  ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]]


In [430]:
EXPORT_MARKERS = True

# Define the parent folder to store the results
OUT_PARENT_FOLDER = f"input/{INPUT_FILE_NAME}_ch{ch_idx}"

if EXPORT_MARKERS:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/markers_{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_markers)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{FR_BAND_FILENAME}_band.npy", final_fr_markers)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{BOTH_BAND_FILENAME}_band.npy", final_hfo_markers)

In [431]:
# To keep the same format as the original data, we will add an extra inner dimension to the final signal
final_ripple_signal = np.array([ripple_seeg_ch])
final_fr_signal = np.array([fr_seeg_ch])
final_hfo_signal = np.array([hfo_seeg_ch])

preview_np_array(final_ripple_signal, "Final Signal")
preview_np_array(final_fr_signal, "Final Signal")
preview_np_array(final_hfo_signal, "Final Signal")

Final Signal Shape: (1, 245760).
Preview: [[ 4.03932046e-06  5.89366126e-05  4.14094388e-04  1.86753797e-03
   6.06652167e-03 ... -6.96934261e-01 -7.47039902e-01 -7.38526318e-01
  -6.11078540e-01 -3.64342973e-01]]
Final Signal Shape: (1, 245760).
Preview: [[ 7.81695139e-05  5.46696135e-04  1.08279173e-03 -1.40195351e-03
  -8.90032230e-03 ... -4.42790739e-01 -4.09728636e-01  5.01444239e-01
   9.77458407e-01 -4.49385585e-02]]
Final Signal Shape: (1, 245760).
Preview: [[ 0.00344332  0.0322592   0.12941021  0.27662443  0.28465375 ...
   0.34871041 -1.59826325 -2.93470791 -2.45221796 -1.1562866 ]]


In [432]:
EXPORT_SIGNAL = True
if EXPORT_SIGNAL:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_signal)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/{FR_BAND_FILENAME}_band.npy", final_fr_signal)
    np.save(f"{OUT_PARENT_FOLDER}/{BOTH_BAND_FILENAME}_band.npy", final_hfo_signal)